# (Core) Project 4 

For this Project, I am starting an additional portfolio project that I will continue to build on over
## Part 1

> For Part 1, I am downloading and preparing a crime dataset for time series analysis. I will then answer provided stakeholder questions and support my answers with visualizations.

- Q1: Comparing Police Districts:
    - Which district has the most crimes? Which has the least?
- Q2: Crimes Across the Years:
    - Is the total number of crimes increasing or decreasing across the years?
    - Are there any individual crimes that are doing the opposite (e.g decreasing when overall crime is increasing or vice-versa)?

### Imports 

In [1]:
#General
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
# Holidays
import holidays
import datetime as dt
from holidays import country_holidays


## Run the cell below to attempt to programmatically find your crime file
import os,glob
## (Optional) SET THE FOLDER FOR FINAL FILES
OUTPUT_FOLDER = 'Data/Chicago/'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)


### Loading Data form local files

In [2]:
# get list of files from folder
crime_files = sorted(glob.glob(OUTPUT_FOLDER+"*.csv"))
df = pd.concat([pd.read_csv(f) for f in crime_files])
df.head(4)

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
0,1323184,01/01/2001 01:00:00 PM,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,2213,22.0,NaN,41.707671,-87.666996
1,1316324,01/01/2001 01:00:00 PM,THEFT,OVER $500,STREET,False,False,1513,15.0,NaN,41.869008,-87.773947
2,1311269,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,421,4.0,NaN,41.756690,-87.561625
3,1323691,01/01/2001 01:00:00 AM,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,False,False,1135,11.0,NaN,41.866694,-87.688513


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7767374 entries, 0 to 56465
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                bool   
 6   Domestic              bool   
 7   Beat                  int64  
 8   District              float64
 9   Ward                  float64
 10  Latitude              float64
 11  Longitude             float64
dtypes: bool(2), float64(4), int64(2), object(4)
memory usage: 666.7+ MB


### Setting Date Index

In [4]:
#Converting Date to datetime object 
df['DateTime'] = pd.to_datetime(df['Date'])

In [5]:
#Setting DateTime Index
df = df.set_index('DateTime')

In [6]:
df.index

DatetimeIndex(['2001-01-01 13:00:00', '2001-01-01 13:00:00',
               '2001-01-01 01:00:00', '2001-01-01 01:00:00',
               '2001-01-01 01:00:00', '2001-01-01 13:00:00',
               '2001-01-01 13:00:00', '2001-01-01 01:00:00',
               '2001-01-01 01:00:00', '2001-01-01 01:00:00',
               ...
               '2023-03-28 00:42:00', '2023-03-28 12:43:00',
               '2023-03-28 00:45:00', '2023-03-28 00:46:00',
               '2023-03-28 00:46:00', '2023-03-28 12:51:00',
               '2023-03-28 12:55:00', '2023-03-28 12:55:00',
               '2023-03-28 12:55:00', '2023-03-28 12:59:00'],
              dtype='datetime64[ns]', name='DateTime', length=7767374, freq=None)

In [7]:
# Renaming feature DateTime to Date for convinience
df.rename(columns = {'DateTime':'Date'}, inplace = True)

### Building Datframe with Holidays

In [8]:
# Setting Region Global Variables
STATE = 'IL'
COUNTRY = 'US'

In [9]:
## making a date range that covers full dataset
all_days = pd.date_range(df["Date"].min(), df["Date"].max())
all_days

DatetimeIndex(['2001-01-01 01:00:00', '2001-01-02 01:00:00',
               '2001-01-03 01:00:00', '2001-01-04 01:00:00',
               '2001-01-05 01:00:00', '2001-01-06 01:00:00',
               '2001-01-07 01:00:00', '2001-01-08 01:00:00',
               '2001-01-09 01:00:00', '2001-01-10 01:00:00',
               ...
               '2022-12-22 01:00:00', '2022-12-23 01:00:00',
               '2022-12-24 01:00:00', '2022-12-25 01:00:00',
               '2022-12-26 01:00:00', '2022-12-27 01:00:00',
               '2022-12-28 01:00:00', '2022-12-29 01:00:00',
               '2022-12-30 01:00:00', '2022-12-31 01:00:00'],
              dtype='datetime64[ns]', length=8035, freq='D')

In [10]:
## Create an instance of the US country holidays.
c_holidays = country_holidays('US')
c_holidays

holidays.country_holidays('US')

In [11]:
## Testing first date
print(all_days[0])
c_holidays.get(all_days[0])

2001-01-01 01:00:00


"New Year's Day"

In [12]:
# For a specific subdivisions (e.g. state or province):
r_holidays = country_holidays(COUNTRY, subdiv=STATE)
r_holidays

holidays.country_holidays('US', subdiv='IL')

In [13]:
## Saving both holiday types as columns
df[f"{COUNTRY} Holiday"] = [c_holidays.get(day) for day in df['Date']]
df[f"{STATE} Holiday"] = [r_holidays.get(day) for day in df['Date']]
df.head()

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude,US Holiday,IL Holiday
DateTime,,,,,,,,,,,,,,
2001-01-01 13:00:00,1323184,01/01/2001 01:00:00 PM,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,2213,22.0,NaN,41.707671,-87.666996,New Year's Day,New Year's Day
2001-01-01 13:00:00,1316324,01/01/2001 01:00:00 PM,THEFT,OVER $500,STREET,False,False,1513,15.0,NaN,41.869008,-87.773947,New Year's Day,New Year's Day
2001-01-01 01:00:00,1311269,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,421,4.0,NaN,41.756690,-87.561625,New Year's Day,New Year's Day
2001-01-01 01:00:00,1323691,01/01/2001 01:00:00 AM,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,False,False,1135,11.0,NaN,41.866694,-87.688513,New Year's Day,New Year's Day
2001-01-01 01:00:00,1422085,01/01/2001 01:00:00 AM,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,True,1023,10.0,NaN,41.858050,-87.695513,New Year's Day,New Year's Day


In [14]:
## Saving a binary is holiday feature
df['Is_Holiday'] = df[f"{COUNTRY} Holiday"].notna()
df['Is_Holiday'].value_counts()



False    7537578
True      229796
Name: Is_Holiday, dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7767374 entries, 2001-01-01 13:00:00 to 2023-03-28 12:59:00
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                bool   
 6   Domestic              bool   
 7   Beat                  int64  
 8   District              float64
 9   Ward                  float64
 10  Latitude              float64
 11  Longitude             float64
 12  US Holiday            object 
 13  IL Holiday            object 
 14  Is_Holiday            bool   
dtypes: bool(3), float64(4), int64(2), object(6)
memory usage: 792.6+ MB


In [16]:
df.head()

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude,US Holiday,IL Holiday,Is_Holiday
DateTime,,,,,,,,,,,,,,,
2001-01-01 13:00:00,1323184,01/01/2001 01:00:00 PM,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,2213,22.0,NaN,41.707671,-87.666996,New Year's Day,New Year's Day,True
2001-01-01 13:00:00,1316324,01/01/2001 01:00:00 PM,THEFT,OVER $500,STREET,False,False,1513,15.0,NaN,41.869008,-87.773947,New Year's Day,New Year's Day,True
2001-01-01 01:00:00,1311269,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,421,4.0,NaN,41.756690,-87.561625,New Year's Day,New Year's Day,True
2001-01-01 01:00:00,1323691,01/01/2001 01:00:00 AM,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,False,False,1135,11.0,NaN,41.866694,-87.688513,New Year's Day,New Year's Day,True
2001-01-01 01:00:00,1422085,01/01/2001 01:00:00 AM,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,True,1023,10.0,NaN,41.858050,-87.695513,New Year's Day,New Year's Day,True


> Removing few columns that will not affect the analysis: 'Ward', 'Latitude', 'Longitude', and 'ID'

In [18]:
df.drop(['Ward', 'Latitude', 'Longitude', 'ID'], axis=1, inplace=True)

In [ ]:
df.to_csv("./Data/ill_crimes_processed.csv.gz", 
           index=False, 
           compression="gzip")